In [1]:
import sqlite3
import cv2
from cv2 import aruco
# from pyzbar.pyzbar import decode
import numpy as np
import pandas as pd
from collections import deque

NO_OF_CAMERA = 0
PATH_TO_DB = "./MARKER_POSITION.db" 

In [ ]:
# カメラを初期化（カメラデバイスID 0を指定）
cap = cv2.VideoCapture(0)

# カメラの初期化が成功したか確認
if not cap.isOpened():
    print("カメラが見つかりません")
    exit()
W, H = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 高解像度に設定
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)  # カメラの実際の解像度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H) # カメラの実際の解像度    
# 
# scale = 1
# 
# # ウィンドウサイズのピクセル数を算出
# width_pixels = int(W * scale)
# height_pixels = int(H * scale)
# 
# # リサイズ可能なウィンドウを作成し、物理サイズに近い表示サイズに設定
# cv2.namedWindow('Camera', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Camera', height_pixels, width_pixels)

# リアルタイム表示
while True:
    # カメラからフレームを読み込む
    ret, frame = cap.read()

    # フレームの読み込みが成功したか確認
    if not ret:
        print("フレームを読み込めません")
        break

    # フレームを表示
    cv2.imshow('Camera', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# カメラリソースを解放し、ウィンドウを閉じる
cap.release()
cv2.destroyAllWindows()


In [2]:
#投影変換座標取得
def setting_square(cap):
    # カメラの初期化 (0はデフォルトのカメラ)
    # cap = cv2.VideoCapture(NO_OF_CAMERA)
    
    # グローバル変数として使用
    points = []
    # マウスコールバック関数
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
            # リサイズ後のクリック座標を元の画像の座標に変換
            points.append((x, y))
            print(f"選択した点: {len(points)}: ({x}, {y})")
            if len(points) == 4:
                cv2.destroyWindow("Select Points")
    # リアルタイムでカメラ映像を表示
    # print("sを押して写真を撮影してください。")
    # while True:
    #     ret, frame = cap.read()
    #     if not ret:
    #         print("カメラから画像を取得できませんでした。")
    #         break
    #     # カメラ映像を表示
    #     cv2.imshow("Webcam - Press any key to capture", frame)
    #     # 'q'キーで撮影せずに終了
    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         print("終了します。")
    #         cap.release()
    #         cv2.destroyAllWindows()
    #         exit()
    #     # 任意のキーを押すと撮影
    #     if cv2.waitKey(1) & 0xFF == ord('s'):
    #         print("写真を撮影しました。")
    #         break
    
    # 静的ver
    ret, frame = cap.read()
    # マウスで4点を選択
    print("左上、右上、右下、左下の順で4つの点をクリックしてください。")
    cv2.imshow("Select Points", frame)
    cv2.setMouseCallback("Select Points", select_point)
    # 4点が選ばれるまで待機
    while len(points) < 4:
        cv2.imshow("Select Points", frame)
        cv2.waitKey(1)
    # 選択された4つの点を取得
    top_left, top_right, bottom_right, bottom_left = points
    # 座標の表示
    print(f"左上: {top_left}")
    print(f"右上: {top_right}")
    print(f"右下: {bottom_right}")
    print(f"左下: {bottom_left}")
    # 選択したポイントに色付き円を描画して確認
    cv2.circle(frame, top_left, 5, (0, 255, 0), -1)
    cv2.circle(frame, top_right, 5, (255, 0, 0), -1)
    cv2.circle(frame, bottom_right, 5, (255, 255, 0), -1)
    cv2.circle(frame, bottom_left, 5, (0, 0, 255), -1)
    # 結果を表示
    cv2.imshow("Selected Points", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # リソースの解放
    cap.release()
    return top_left, top_right, bottom_right, bottom_left
# 透視変換関数の定義
def PerspectiveTransform(image, W, H, tl, tr, br, bl):
    # 視点の表裏を逆転
    src_pts = np.array([tr, tl, bl, br], dtype='float32')
    dst_pts = np.array([(0, 0), (W, 0), (W, H), (0, H)], dtype='float32')
    # 変換行列を取得し変換を適用
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (W, H))
    return warped

def PerspectiveTransform_2(image, W, H, tl, tr, br, bl):
    # 入力画像の4隅の座標
    src_pts = np.array([tl, tr, br, bl], dtype='float32')
    # 出力画像の4隅の座標を180度回転させた位置に設定
    dst_pts = np.array([(W, H), (0, H), (0, 0), (W, 0)], dtype='float32')
    # 変換行列を取得し変換を適用
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (W, H))
    return warped

def update_positions(path_to_db, positions):
    """
    positionsテーブルを新しいデータで上書きする関数。

    :param path_to_db: データベースファイルへのパス
    :param positions: (id, x_rel, y_rel)のタプルからなるリスト
    """
    conn = None
    cursor = None
    try:
        # データベースに接続
        conn = sqlite3.connect(path_to_db)
        cursor = conn.cursor()

        # 既存のpositionsテーブルを削除
        cursor.execute("DROP TABLE IF EXISTS positions")

        # 新しいpositionsテーブルを作成
        cursor.execute("""
            CREATE TABLE positions (
                id TEXT PRIMARY KEY,
                x_rel INTEGER,
                y_rel INTEGER
            )
        """)

        # 新しいデータを挿入
        cursor.executemany("""
            INSERT INTO positions (id, x_rel, y_rel)
            VALUES (?, ?, ?)
        """, positions)

        # 変更をコミット
        conn.commit()
        print("positionsテーブルを新しいデータで更新しました。")

    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
    finally:
        # リソースの解放
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [6]:
# コントラストと明るさの変更
alpha = 1.5  # コントラストの倍率（1より大きい値でコントラストが上がる）
beta = 0  # 明るさの調整値（正の値で明るくなる）

# カメラの設定
cap = cv2.VideoCapture(NO_OF_CAMERA)

# 高解像度に設定
W,H = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(f"W:{W} H:{H}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)  # カメラの実際の解像度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H) # カメラの実際の解像度 

# # リサイズ可能なウィンドウを作成し、サイズを指定
# cv2.namedWindow("QR Code Detection", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("QR Code Detection", 1290, 1290)

if not cap.isOpened():
    print("カメラが見つかりませんでした。")
    exit()
# グリッドのサイズと透視変換後のサイズを設定
W, H = 1000, 1000  # 透視変換後の幅と高さ
N, L = 10, 10  # グリッドの行数と列数
# 透視変換のための四隅の座標を取得
top_left, top_right, bottom_right, bottom_left = setting_square(cap)



# ArUcoの辞書とパラメータの設定
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters()

print("カメラ映像の表示を開始します。'q'キーで終了します。")

cap = cv2.VideoCapture(NO_OF_CAMERA)

# databaseに接続
conn = sqlite3.connect(PATH_TO_DB)
while True:
    ret, frame = cap.read()
    if not ret:
        print("フレームを取得できませんでした。")
        break
        
    # フレームをトリム
    parsed_frame = PerspectiveTransform_2(frame, W, H, top_left, top_right, bottom_right, bottom_left)

    # グレースケールに変換
    gray = cv2.cvtColor(parsed_frame, cv2.COLOR_BGR2GRAY)

    # マーカーの検出
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    if ids is not None:
        # 検出されたマーカーの中心座標を計算して表示
        positions = []
        for i, corner in enumerate(corners):
            # 四隅の座標を取得
            points = corner[0]
            # 中心座標を計算
            center_x = int(np.mean(points[:, 0]))
            center_y = int(np.mean(points[:, 1]))
            center = (center_x, center_y)
            id = ids[i][0]
            print(f"マーカーID: {id} の中心座標: {center}")
            
            # 中心に円を描画
            cv2.circle(parsed_frame, center, 5, (0, 255, 0), -1)
            # マーカーIDを表示
            cv2.putText(parsed_frame, f"ID: {ids[i][0]}", (center_x - 10, center_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # データベースに書き込む
            positions.append((id,center_x,center_y))
        update_positions(PATH_TO_DB,positions)

    # 検出されたマーカーをフレームに描画
    detected_frame = aruco.drawDetectedMarkers(parsed_frame, corners, ids)

    # フレームを表示
    cv2.imshow('Transformed Frame', detected_frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# リソースの解放
cap.release()
cv2.destroyAllWindows()

# データベースを閉じる

W:1280.0 H:720.0
左上、右上、右下、左下の順で4つの点をクリックしてください。
選択した点: 1: (62, 65)
選択した点: 2: (652, 74)
選択した点: 3: (649, 663)
選択した点: 4: (56, 658)
左上: (62, 65)
右上: (652, 74)
右下: (649, 663)
左下: (56, 658)
カメラ映像の表示を開始します。'q'キーで終了します。
マーカーID: 1 の中心座標: (877, 836)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (877, 836)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (810, 831)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (787, 812)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (787, 812)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (784, 793)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (782, 774)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 766)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 766)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (781, 766)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 767)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 767)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 767)
positionsテーブルを新しいデータで更新しました。
マーカーID: 1 の中心座標: (780, 767)
positionsテーブルを新しいデー

In [23]:
# sqlite3 テーブル作成
conn = sqlite3.connect(PATH_TO_DB)
cursor = conn.cursor()
cursor.execute("""DROP TABLE IF EXISTS map_bounds """)
cursor.execute("""
CREATE TABLE map_bounds (
    id INTEGER ,
    top_left_lat REAL NOT NULL,
    top_left_lng REAL NOT NULL,
    top_right_lat REAL NOT NULL,
    top_right_lng REAL NOT NULL,
    bottom_left_lat REAL NOT NULL,
    bottom_left_lng REAL NOT NULL,
    bottom_right_lat REAL NOT NULL,
    bottom_right_lng REAL NOT NULL)
""")
conn.commit()
conn.close()

In [5]:
# チェック用
conn = sqlite3.connect(r"C:\Users\tora2\IdeaProjects\cityScope\visual\screen_grid\MARKER_POSITION.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM positions")
for r in cursor.fetchall():
    print(r)
conn.close()

(b'\x01\x00\x00\x00', 908, 907)
